In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from collections import deque
from sklearn import preprocessing

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data Processing

In [2]:
# df = pd.read_csv('DenoisngData.csv', index_col=0)
# df.drop('timestemp', axis = 1, inplace=True)

In [3]:
df = pd.read_csv('/Users/tirshfield/Documents/Education/Graduate/2018-2019/Fall_2018/Deep_Learning/Final_Project/Saved/Data/attempt_1/100_clean.csv', index_col=0)
df.drop(['Ticker', 'Date', 'Adjusted Close', 'Volume'], axis = 1, inplace=True)

In [4]:
X_train_total = []
Y_train_total = []

X_test_total = []
Y_test_total = []

seq_len = 12

keys = list(df['gvkey'].unique())
keys = [key for key in keys if key not in [8214, 15350]]

used_keys = []

for key in keys:
    df_temp = df[df['gvkey']==key].drop('gvkey', axis=1) 
    if len(df_temp) != 115:
        print(f'we drop the {key}')
        continue
    
    used_keys.append(key)
        
    y_1 = df_temp['EBIT/EV']
    y_0 = df_temp['EBIT/EV'].shift(1)
    df_temp['return'] = (y_1-y_0)/pd.DataFrame([y_1, y_0]).T.abs().max(axis=1)
#     df_temp['return'] = (y_1-y_0)/np.mean(np.abs(y_1), np.abs(y_0))
#     df_temp['return'] = (df_temp['EBIT/EV']-df_temp['EBIT/EV'].shift(1))/df_temp['EBIT/EV'].shift(1)
    df_temp.drop('EBIT/EV', axis=1, inplace = True) 
    df_temp.dropna(inplace=True)
    col_names = df_temp.columns.values
    min_max_scaler = preprocessing.MinMaxScaler()
    df_temp.loc[:, col_names[:-1]] = min_max_scaler.fit_transform(df_temp.loc[:, col_names[:-1]])
    df_temp.ffill(inplace=True)
    
    X_list = []
    Y_list = []
    for i in range(len(df_temp)-seq_len-1):
        X_list += [df_temp.iloc[i:i+seq_len].values]
        Y_list += [df_temp.iloc[i+seq_len+1, -1]]

    X = np.array(X_list)
    Y = np.array(Y_list)

    # First 72 quarters are for training
    if X.shape[0] >= 5: # Some gvkeys have no insufficient data (dont have 12 quarters of data).
                              # >=5 to ensure there is testing data.
        X_train = X[:int(len(X)*0.7)]
        X_test = X[int(len(X)*0.7):]

        Y_train = Y[:int(len(X)*0.7)]
        Y_test = Y[int(len(X)*0.7):]

        X_train_total += [X_train]
        X_test_total += [X_test]

        Y_train_total += [Y_train]
        Y_test_total += [Y_test]

we drop the 2698
we drop the 5680
we drop the 5709
we drop the 5903
we drop the 6135
we drop the 8092
we drop the 10565
we drop the 11012
we drop the 11017
we drop the 12304
we drop the 12785
we drop the 13400
we drop the 13679
we drop the 17175
we drop the 17422
we drop the 19602
we drop the 21807
we drop the 21824
we drop the 22964
we drop the 23109
we drop the 23337
we drop the 24552
we drop the 25740
we drop the 25750
we drop the 28347
we drop the 30241
we drop the 32796
we drop the 33047
we drop the 33048
we drop the 33068
we drop the 33081
we drop the 33150
we drop the 33279
we drop the 33652
we drop the 105394
we drop the 107154
we drop the 107245
we drop the 107712
we drop the 107781
we drop the 108768
we drop the 108831
we drop the 115044
we drop the 134305
we drop the 143770
we drop the 146659
we drop the 148239
we drop the 149683
we drop the 162453
we drop the 162914
we drop the 163596
we drop the 170920
we drop the 174428
we drop the 185323
we drop the 185689
we drop the 18

In [5]:
X_train_total = np.vstack(X_train_total)
X_test_total = np.vstack(X_test_total)

Y_train_total = np.concatenate(Y_train_total)
Y_test_total = np.concatenate(Y_test_total)

In [6]:
for i in [X_train_total, X_test_total, Y_train_total, Y_test_total]:
    print(f'shape is {i.shape}')

shape is (2940, 12, 17)
shape is (1302, 12, 17)
shape is (2940,)
shape is (1302,)


## Model 

In [7]:
tf.reset_default_graph()

hm_epochs = 200
n_classes = 1
n_batches = 4
batch_size = X_train.shape[0] / n_batches
chunk_size = X_train.shape[2]
rnn_size = 64
num_layers = 1

x = tf.placeholder('float', [None, seq_len, chunk_size])
y = tf.placeholder('float')

In [8]:
def recurrent_neural_network(x):
    layer = {'weights': tf.Variable(tf.random_normal([rnn_size, n_classes])),
             'biases': tf.Variable(tf.random_normal([n_classes]))}

#     layer = {'weights':tf.Variable(np.random.normal(size=(rnn_size,n_classes)).astype('float32')),
#              'biases':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, seq_len, 0)

    lstm_cells = []
    
    for li in range(num_layers):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, state_is_tuple=True, initializer=tf.contrib.layers.xavier_initializer())
        dropout_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, input_keep_prob=1.0, output_keep_prob=1.0)
        lstm_cells.append(dropout_cell)
        
    multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)

    outputs, states = rnn.static_rnn(multi_cell, x, dtype=tf.float32)
    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']  # softmax layer

#     attention_weights = tf.Variable(np.append(np.random.normal(size=seq_len-1), 5))
#     attention_weights = tf.Variable(tf.random_normal([seq_len]))
#     attention_ouput = tf.reduce_sum(tf.multiply(tf.convert_to_tensor(outputs) , tf.Variable(tf.random_normal([seq_len]))[:, tf.newaxis, tf.newaxis]), axis=0)
#     output = tf.matmul(attention_ouput,layer['weights']) + layer['biases'] # softmax layer

    return output

In [9]:
def train_neural_network(x, X_train, X_test, Y_train, Y_test):
    prediction = recurrent_neural_network(x)
    cost = tf.losses.mean_squared_error(predictions=prediction, labels=y)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(n_batches):
                epoch_x = X_train[i * int(batch_size):(i + 1) * int(batch_size)]
                epoch_y = Y_train[i * int(batch_size):(i + 1) * int(batch_size)]

                _, c = sess.run([optimizer, cost], feed_dict={
                                x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',
                  hm_epochs, 'loss:', np.mean(epoch_loss))

        pred_train = sess.run(prediction, feed_dict={
                              x: X_train.astype('float32')})[:, 0]
        pred_test = sess.run(prediction, feed_dict={
                             x: X_test.astype('float32')})[:, 0]

        print('Testing MAE:', np.mean(np.abs(pred_test - Y_test)))
        print('Testing MSE:', np.mean((pred_test - Y_test)**2))
        
        return pred_test

pred_result = train_neural_network(x, X_train_total, X_test_total, Y_train_total, Y_test_total)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 200 loss: 4.551334157586098
Epoch 1 completed out of 200 loss: 1.3708917126059532
Epoch 2 completed out of 200 loss: 1.4891034662723541
Epoch 3 completed out of 200 loss: 1.2839937135577202
Epoch 4 completed out of 200 loss: 0.9811406880617142
Epoch 5 completed out of 200 loss: 0.9300076141953468
Epoch 6 completed out of 200 loss: 0.8511447533965111
Epoch 7 completed out of 200 loss: 0.890371523797512
Epoch 8 completed out of 200 loss: 0.8153553828597069
Epoch 9 completed out of 200 loss: 0.8298826962709427
Epoch 10 completed out of 200 loss: 0.8119854629039764
Epoch 11 completed out of 200 loss: 0.7989329472184181
Epoch 12 completed out of 200 loss: 0.7919664084911346
Epoch 13 completed out of 200 loss: 0.7910673171281815
Epoch 14 completed out of 200 loss: 0.7889525443315506
Epoch 15 completed out of 200 loss: 0.784821055829525
Epoch 16 completed out of 200 loss: 0.7840394303202629
Epoc

Epoch 145 completed out of 200 loss: 0.7695092856884003
Epoch 146 completed out of 200 loss: 0.7804016619920731
Epoch 147 completed out of 200 loss: 0.8073627725243568
Epoch 148 completed out of 200 loss: 0.8633501678705215
Epoch 149 completed out of 200 loss: 0.9864927381277084
Epoch 150 completed out of 200 loss: 1.1144966632127762
Epoch 151 completed out of 200 loss: 1.2072824537754059
Epoch 152 completed out of 200 loss: 0.9329774081707001
Epoch 153 completed out of 200 loss: 0.8320029452443123
Epoch 154 completed out of 200 loss: 0.8343302235007286
Epoch 155 completed out of 200 loss: 0.8200704827904701
Epoch 156 completed out of 200 loss: 0.7708918005228043
Epoch 157 completed out of 200 loss: 0.787386953830719
Epoch 158 completed out of 200 loss: 0.7760450839996338
Epoch 159 completed out of 200 loss: 0.7666136994957924
Epoch 160 completed out of 200 loss: 0.7705319598317146
Epoch 161 completed out of 200 loss: 0.7646416872739792
Epoch 162 completed out of 200 loss: 0.7644270583

In [15]:
#Splits pred_result into blocks of 31
pred_result_split = np.split(pred_result, len(pred_result)/31)

#Splits Y_test_total into blocks of 31
Y_test_total_split = np.split(Y_test_total, len(Y_test_total)/31)

#Match both with GVKEY
pred_result_dict = dict(zip(used_keys, pred_result_split))
Y_test_total_dict = dict(zip(used_keys, Y_test_total_split))

{1209: array([-0.05140162, -0.06535172, -0.04389203, -0.05804062, -0.0586493 ,
       -0.06211996, -0.06196535, -0.07251   , -0.0865922 , -0.07164419,
       -0.05488122, -0.02088034, -0.00209332,  0.00915468,  0.01245952,
        0.01620722,  0.0592165 ,  0.0586642 ,  0.04761219,  0.06040835,
        0.06804442,  0.03215551,  0.05029726,  0.09717786,  0.04959249,
       -0.02446353, -0.26150584, -0.3680581 , -0.4440875 , -0.48023665,
       -0.5080085 ], dtype=float32), 1380: array([-1.2713854 , -1.2584085 , -1.233273  , -1.2098687 , -1.1611562 ,
       -1.1108499 , -0.97406983, -0.91063166, -0.9210136 , -0.88919353,
       -0.8890555 , -0.9436276 , -1.0168433 , -1.1009424 , -1.1334202 ,
       -1.1714327 , -1.2539389 , -1.2951422 , -1.2874985 , -1.1943564 ,
       -1.1715107 , -1.1039567 , -1.0608752 , -0.98736787, -0.95781565,
       -0.68777204, -0.6800401 , -0.6264763 , -0.6666093 , -0.7005806 ,
       -0.7560463 ], dtype=float32), 1678: array([-1.6962409 , -1.6316929 , -1.5767062